<h1><center>Laboratorio 3: Clasificación y Clustering 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Roa
- Nombre de alumno 2: Matías Sepúlveda


### **Link de repositorio de GitHub:** `https://github.com/jroallanos/LDPCPCDD19`



### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Manejo de DataFrames con la libreria `pandas`.
- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar y aprovechar las ventajas que nos ofrece la libreria `pandas`.
- Utilizar `plotly` para obtener información gráfica del dataset.
- Aplicar las ventajas que nos ofrece crear un pipeline.
- Visualizar el funcionamiento de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

     |████████████████████████████████| 13.2MB 230kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 1.2MB 32.2MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=45c3dd6d8404a412a71c9442f81e55ecf14804ec59edcfe0ea0b01c5ee226b1f
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51362 sha256=e30565e854cf73ba9ef7314798a92fc948a3452b5fa432247c92e426318942a1
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent


# 1. Predicciones futboleras.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

Para este laboratorio se deberá trabajar con los csv ```FullData.csv```, donde deberán aplicar algoritmos de Clustering y clasificación información relevante de jugadores de futbol.

Para comenzar cargue el dataset señalado y visualice a traves de un head los atributos que posee el dataset.

In [3]:
dir = '/content/20210528142421E83200FFF63A69__stats_players_(1).csv'
df_players = pd.read_csv(dir)
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,Short_Pass,Long_Pass,Acceleration,Speed,Stamina,Strength,Balance,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185 cm,80 kg,Right,32,High / Low,4,5,93,92,22,23,31,63,96,29,85,86,84,83,77,91,92,92,80,63,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170 cm,72 kg,Left,29,Medium / Medium,4,4,95,97,13,26,28,48,95,22,90,94,77,88,87,92,87,74,59,95,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174 cm,68 kg,Right,25,High / Medium,5,5,95,96,21,33,24,56,88,36,80,80,75,81,75,93,90,79,49,82,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182 cm,85 kg,Right,30,High / Medium,4,4,91,86,30,38,45,78,93,41,84,83,77,83,64,88,77,89,76,60,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193 cm,92 kg,Right,31,Medium / Medium,4,1,48,30,10,11,10,29,85,30,70,70,15,55,59,58,61,44,83,35,52,78,25,25,13,16,14,11,47,11


De la tabla se reconoce:

1. `Name`: Nombre del jugador
2. `Nationality`: Nacionalidad del jugador
3. `National_Position`: Posición en la selección, este valor solo existe cuando el jugador es seleccionado nacional.
4. `Club_Position`: Posición del jugador. Este parámetro tiene valor en gran parte de los jugadores, existen casos como `Res` o `Sub` que no los deberá considerar en la respectiva sección.
5. `Height`: Altura del jugador en centímetros. Si nota esta variable tiene un string, transforme a int.
6. `Weight`: Peso del jugador en kilogramos. Si nota esta variable tiene un string, transforme a int.
7. `Preffered_Foot`: Pie con que prefiere patear.
8. `Age`: Edad
9. `Work_Rate`: Frecuencia con que trabaja.

Mientras que el resto de las características representan estadísticas personales (de acuerdo a la habilidad) para cada uno de los jugadores.


## 1.1 Predicción de seleccionados nacionales. (2 puntos)

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


### 1.1.1 Generación de Labels para la Clasificación

Tras el trágico despido de la mítica mascota de la población de Maipú, *Renacín*, decide adentrarse en la predicción futbolera. Para esto, utilizando los datos cargados en las celdas anteriores, le consulta si es posible predecir **si un jugador de football será seleccionado nacional en base a sus estadísticas**

En este primer experimento de clasificación comience generando los labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. Hecho esto, ¿Como se distribuyen los datos para cada clase?, comente lo que observa.


In [4]:
df_players["National_Position"].fillna(0, inplace=True)
df_players['National_Position'].value_counts()

0      16513
Sub      556
LCB       48
GK        47
RCB       46
LB        39
RB        38
RM        34
LM        32
ST        30
RCM       25
LCM       25
LDM       19
CAM       19
LS        18
RDM       18
RS        18
CM         9
CB         9
CDM        9
LW         7
RW         7
RAM        4
LWB        4
LAM        4
RWB        4
LF         3
RF         3
Name: National_Position, dtype: int64

In [5]:
df_players['National_Position'] = df_players['National_Position'].replace({'Sub':1, 'LCB':1,  'GK':1, 'RCB':1,  'LB':1,  'RB':1,  'RM':1,  'LM':1,  'ST':1,
       'RCM':1, 'LCM':1, 'LDM':1, 'CAM':1,  'LS':1,  'RS':1, 'RDM':1,  'CB':1, 'CDM':1,  'CM':1, 'LW':1,  
       'RW':1, 'RAM':1, 'LWB':1, 'LAM':1, 'RWB':1,  'RF':1, 'LF':1})

In [6]:
print(f"Cantidad de datos etiquetados como seleccionados (1): {(df_players.National_Position == 1).sum()}")
print(f"Cantidad de datos etiquetados como no-seleccionados (0): {(df_players.National_Position == 0).sum()}")

Cantidad de datos etiquetados como seleccionados (1): 1075
Cantidad de datos etiquetados como no-seleccionados (0): 16513


### 1.1.2 Camino a la Clasificación

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="300">
</p>



Si bien los atributos de interés se encuentran en su mayoría limpios, existen valores mal ingresados al `DataFrame`. Para esto: 

1. Transforme a enteros todos los valores numéricos de la tabla que considere pertinentes. Por ejemplo: El peso aparece como strings y además estos continene la unidad. En este caso, debería transformar estos valores a enteros. Puede usar directamente funciones sobre el DataFrame (como `apply`) o el submódulo de las Series `str`.
2. Luego, aplique un ```ColumnTransformer``` para pre-procesar los datos del DataFrame de la siguiente forma:

Aplicar ```OneHotEncoder()``` sobre los atributos:

```python
['Preffered_Foot', 'Work_Rate']
```

Aplicar ```MinMaxScaler()``` sobre los atributos:

```python
['Age']
```

Aplicar ```StandardScaler()``` sobre los atributos:

```python
['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
```

**Nota 2**: Puede copiar/pegar directamente las columnas en el `ColumnTransformer`.


Hecho lo anterior, defina al menos 3 pipelines para la clasificación donde anexe las transformaciones señaladas variando entre cada pipeline solo los clasificadores (puede escoger cualquiera de los clasificadores vistos en clases).

**To-Do:**

- [ ] Modificar valores mal ingresados.
- [ ] Aplicar ```ColumnTransformer``` sobre los datos a través de un pipeline.
- [ ] Justificar porque se utilizan estos pre-procesamientos.
- [ ] Crear 3 pipelines con diferentes clasificadores.
- [ ] No entrenar los datos aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [7]:
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,Short_Pass,Long_Pass,Acceleration,Speed,Stamina,Strength,Balance,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,1,LW,185 cm,80 kg,Right,32,High / Low,4,5,93,92,22,23,31,63,96,29,85,86,84,83,77,91,92,92,80,63,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,1,RW,170 cm,72 kg,Left,29,Medium / Medium,4,4,95,97,13,26,28,48,95,22,90,94,77,88,87,92,87,74,59,95,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,1,LW,174 cm,68 kg,Right,25,High / Medium,5,5,95,96,21,33,24,56,88,36,80,80,75,81,75,93,90,79,49,82,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,1,ST,182 cm,85 kg,Right,30,High / Medium,4,4,91,86,30,38,45,78,93,41,84,83,77,83,64,88,77,89,76,60,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,1,GK,193 cm,92 kg,Right,31,Medium / Medium,4,1,48,30,10,11,10,29,85,30,70,70,15,55,59,58,61,44,83,35,52,78,25,25,13,16,14,11,47,11


In [8]:
df_players['Height'] = df_players['Height'].apply(lambda x: x.split(' ')[0])
df_players["Height"]=df_players["Height"].astype(int)

df_players['Weight'] = df_players['Weight'].apply(lambda x: x.split(' ')[0])
df_players["Weight"]=df_players["Weight"].astype(int)

In [9]:
preprocessing_transformer = ColumnTransformer(
    transformers = [
        ('OneHotEncoder', OneHotEncoder(), ['Preffered_Foot', 'Work_Rate']),
        ('MinMaxScaler', MinMaxScaler(), ['Age']), 
        ('StandardScaler', StandardScaler(), ['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
                                              'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 
                                              'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 
                                              'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys'])]) 

Justificación:

> Se utiliza el OneHotEncoder sobre las columnas Preffered_Foot y Work_Rate porque las necesitamos de forma numérica para incorporarlas al clustering, dado que los métodos suelen utilizar solo variables numéricas.

> Para el MinMaxScaler(), se usa porque se necesita "normalizar" la edad a un rango entre 0 y 1, para que así las edades más grandes no pesen excesivamente en el modelo a desarrollar. Esta codificación es lineal y proporcional al rango de la edad de los jugadores, donde la máxima edad pasa a ser el último valor de este nuevo intervalo (1) y la mínima edad representa el otro extremo del nuevo intervalo (0).

> Por último, se utiliza el StandardScaler() para regularizar los valores que serán insertados en un futuro modelo de clasificación. Regula los valores extremos para que no pesen demasiado, y los deja todos en una similar escala.

El resultado de esta codificación es que para el próximo modelo a entrenar, ninguna de las "features" será mayor a 1 ni menor que 0, por lo que el peso de los atributos sólo importará según su capacidad de predecir la variable explicativa.








In [10]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

pipeline1 = Pipeline([('Preprocessing', preprocessing_transformer), ('Modelo', SVC())])
pipeline2 = Pipeline([('Preprocessing', preprocessing_transformer), ('Modelo', GaussianNB())])
pipeline3 = Pipeline([('Preprocessing', preprocessing_transformer), ('Modelo', RandomForestClassifier())])

### 1.1.3 Métricas de evaluación + Opcional

<p align="center">
  <img src="https://media4.giphy.com/media/3orif8r5vpbAdQU3iU/200w.gif?cid=82a1493btc1h43n1h4c59p13kjv6adugso4lo4292zbgt66b&rid=200w.gif&ct=g" width="300">
</p>

Interesado por saber que tan bien funciona su clasificador, Renacin le solicita que construya **a mano** las siguientes métricas de evaluación para el caso de **clasificación binaria**:

- Accuracy (Exactitud):

$$Accuracy = \dfrac{número \, de \, predicciones \, correctas}{número \, de \,predicciones \, totales}$$


- Precision: 

$$Precision = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \, positivos}$$

- Recall:

$$Recall = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \,negativos}$$

<p align="center">
  <img src="https://www.researchgate.net/publication/336402347/figure/fig3/AS:812472659349505@1570719985505/Calculation-of-Precision-Recall-and-Accuracy-in-the-confusion-matrix.ppm" width="350">
</p>

**To-Do:**

- [ ] Crear funciones para las métricas para clasificación binaria: Accuracy, Precision y Recall. 
- [ ] Compruebe el correcto funcionamiento de sus métricas a través de los test.

**Hint:** Utilice confusion_matrix para obtener TP, FP, FN y TN.



In [11]:
def accuracy(y_true, y_pred):
  ##### Código aquí #######
  TN = confusion_matrix(y_true, y_pred)[0][0]
  FP = confusion_matrix(y_true, y_pred)[0][1]
  FN = confusion_matrix(y_true, y_pred)[1][0]
  TP = confusion_matrix(y_true, y_pred)[1][1] 
  return (TP + TN)/(TP+FP+FN+TN)

def presicion(y_true, y_pred):
  ##### Código aquí #######
  TP = confusion_matrix(y_true, y_pred)[1][1]
  FP = confusion_matrix(y_true, y_pred)[0][1]
  if TP+FP == 0:
    return 0
  else:
    return TP/(TP+FP)

def recall(y_true, y_pred):
  ##### Código aquí #######
  TP = confusion_matrix(y_true, y_pred)[1][1]
  FN = confusion_matrix(y_true, y_pred)[1][0]
  if TP+FN==0:
    return 0
  else:
    return TP/(TP+FN)

In [12]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

y_true = [0, 1, 0, 0, 1, 0]
y_pred = [0, 0, 1, 0, 0, 1]

# Tests de metricas
assert accuracy_score(y_true, y_pred) == accuracy(y_true, y_pred)
assert recall_score(y_true, y_pred) == recall(y_true, y_pred)
assert precision_score(y_true, y_pred) == presicion(y_true, y_pred)

### 1.1.4 Entrenemos los pipelines

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="300">
</p>


Entrene en base al pipeline generado en los pasos anteriores, procure separar los datos de entrenamiento en un conjunto de entrenamiento y de prueba (la proporción queda a su juicio). Una vez entrenado, muestre los resultados para cada uno de los clasificadores y comente lo que observa de los resultados obtenidos usando las métricas creadas anteriormente, ¿podrían mejorarse los resultados?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar el pipeline.
- [ ] Utilizar las métricas codificadas anteriormente y analizar los resultados.


In [13]:
# Primero vamos a separar los datos en conjuntos de entrenamiento y de prueba
X = df_players.loc[:,['Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']]
y = df_players.loc[:,['National_Position']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [14]:
pipeline1.fit(X_train, y_train['National_Position'])
pipeline2.fit(X_train, y_train['National_Position'])
pipeline3.fit(X_train, y_train['National_Position'])

Pipeline(memory=None,
         steps=[('Preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('OneHotEncoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(copy=True,

In [15]:
y_pred1 = pipeline1.predict(X_test)
y_pred2 = pipeline2.predict(X_test)
y_pred3 = pipeline3.predict(X_test)

In [16]:
print('Para el primer pipeline con SVM: ')
print('Accuracy: ', accuracy(y_test, y_pred1))
print('Presicion: ', presicion(y_test, y_pred1))
print('Recall: ', recall(y_test, y_pred1))
print()
print('Para el segundo pipeline con Naive Bayes: ')
print('Accuracy: ', accuracy(y_test, y_pred2))
print('Presicion: ', presicion(y_test, y_pred2))
print('Recall: ', recall(y_test, y_pred2))
print()
print('Para el tercer pipeline con Random Forest: ')
print('Accuracy: ', accuracy(y_test, y_pred3))
print('Presicion: ', presicion(y_test, y_pred3))
print('Recall: ', recall(y_test, y_pred3))

Para el primer pipeline con SVM: 
Accuracy:  0.9403069926094372
Presicion:  0
Recall:  0.0

Para el segundo pipeline con Naive Bayes: 
Accuracy:  0.8000758006443055
Presicion:  0.15992647058823528
Recall:  0.5523809523809524

Para el tercer pipeline con Random Forest: 
Accuracy:  0.9435285199924199
Presicion:  0.7297297297297297
Recall:  0.08571428571428572


**Respuesta**:

```
Si observamos los resultados de las métricas, principalmente de presicion y recall, vemos que están bastante bajas, por lo que existe una clase que les está costando predecir, y eso no se puede observar en accuracy, que sufre con clases desbalanceadas y no es capaz de indicar si estamos prediciendo bien o mal. Por lo que evidentemente existe espacio de mejora, y esto recae en predecir de mejor manera la clase positiva.
```

## 1.2 Predicción de posiciones de jugadores. (2 puntos)

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

### 1.2.1 Creación de nuevas labels.



No conforme con el trabajo de la sección 1.1, Renacin les *ordena* que construyan un clasificador nuevo, pero para predecir la posición de los jugadores del dataset. Para lograr esto, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, con esto les recalca que no deben utilizar los valores ```Sub``` y ```Res``` de este atributo, pidiendo que agrupen los valores en los siguientes grupos:

```python
Ataque=['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']
MedioCampistas=['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']
Defensas=['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']
Arquero =['GK']
```

Con lo señalado, Renacin les señala que no deberian utilizar el label utilizado en la sección 1.1, ya que el problema de clasificación ahora es de posiciones.

**To-Do**:

- [ ] Aplique las etiquetas de ```Ataque```, ```MedioCampista```, ```Defensa``` y ```Arquero``` en los valores señalados en esta sección.
- [ ] Muestre como se distribuyen los datos etiquetados.

In [17]:
df_players = df_players.dropna(subset=['Club_Position'])

In [18]:
df_players = df_players[-df_players['Club_Position'].isin(['Sub', 'Res'])]

In [19]:
df_players['Club_Position'] = df_players['Club_Position'].replace({'ST': 'Ataque', 'CAM' : 'Ataque', 'RS' : 'Ataque' , 'LF' : 'Ataque', 'LAM': 'Ataque', 'LS' : 'Ataque', 
                                                                   'RF' : 'Ataque', 'RAM' : 'Ataque', 'CF' : 'Ataque', 'RCM' : 'MedioCampistas', 'LW' : 'MedioCampistas', 
                                                                   'RW' : 'MedioCampistas', 'LCM' : 'MedioCampistas', 'LM' : 'MedioCampistas', 'RM' : 'MedioCampistas', 'CM' : 'MedioCampistas',
                                                                   'LCB' : 'Defensa', 'RB' : 'Defensa', 'RCB' : 'Defensa', 'LDM' : 'Defensa', 'LB' : 'Defensa', 'CDM': 'Defensa',
                                                                   'RDM' : 'Defensa', 'CB' : 'Defensa', 'RWB' : 'Defensa', 'LWB' : 'Defensa', 'GK' : 'Arquero'})

In [20]:
print(f"Cantidad de datos etiquetados como delantero: {(df_players.Club_Position == 'Ataque').sum()}")
print(f"Cantidad de datos etiquetados como defensa: {(df_players.Club_Position == 'Defensa').sum()}")
print(f"Cantidad de datos etiquetados como medio campistas: {(df_players.Club_Position == 'MedioCampistas').sum()}")
print(f"Cantidad de datos etiquetados como arqueros: {(df_players.Club_Position == 'Arquero').sum()}")

Cantidad de datos etiquetados como delantero: 1252
Cantidad de datos etiquetados como defensa: 3185
Cantidad de datos etiquetados como medio campistas: 1880
Cantidad de datos etiquetados como arqueros: 632


### 1.2.2 Entrenar un pipeline

Aplicando el mismo preprocesamiento solicitado en 1.1.2, genere un pipeline con un clasificador a gusto y evalúe al modelo de clasificación utilizando la **librería `scikit-learn`** (esto debido a que este es un problema multi clase y las funciones solicitadas anteriormente era para una clasificación binaria).

De los resultados obtenidos, ¿El modelo pudo clasificar mejor este problema o el de la sección 1.1?, Comente sus resultados.

**To-Do**:

- [ ] Genere un pipeline similar al visto en la sección 1.1.2.
- [ ] Aplique las métricas ofrecidas por la librería scikit-learn.
- [ ] Responda la pregunta del enunciado.

In [21]:
# Primero vamos a separar los datos en conjuntos de entrenamiento y de prueba
X = df_players.loc[:,['Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']]
y = df_players.loc[:,['Club_Position']]
y = y['Club_Position'].replace({'Ataque': 0, 'MedioCampistas' : 1, 'Defensa' : 2, 'Arquero' : 3 }) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [22]:
pipeline4 = Pipeline([('Preprocessing', preprocessing_transformer), ('Modelo', RandomForestClassifier())])
pipeline4.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('Preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('OneHotEncoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(copy=True,

In [23]:
y_pred4 = pipeline4.predict(X_test)
print('Para el cuarto pipeline con Random Forest: ')
print('Accuracy: ', accuracy_score(y_test, y_pred4))
print('Precision: ', precision_score(y_test, y_pred4, average='macro'))
print('Recall: ', recall_score(y_test, y_pred4, average='macro'))

Para el cuarto pipeline con Random Forest: 
Accuracy:  0.8067146282973621
Precision:  0.8287804768319508
Recall:  0.8084022816120588


**Respuesta**:

```
Definitivamente se pudo clasificar mejor este problema, si bien el accuracy es menor, en este caso no nos importa tanto ya que antes las métricas precision y recall eran horribles en el modelo anterior, y en este modelo son sustancialmente mejores, por lo que en general es capaz de predecir mejor las diferentes clases aunque no estén balanceadas.
```

## 1.3 Clustering Sobre el Dataset (2 puntos)

<p align="center">
  <img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1832816%2F349289c622faaff3afc517a46fcf4c71%2Fk-means.jpg?generation=1577258176400530&alt=media" width="200">
</p>

### 1.3.1 Elección de numero de clusters

<p align="center">
  <img src="https://media1.tenor.com/images/a85e9df2f5784d6513dce853ba9d5755/tenor.gif?itemid=18229271" width="200">
</p>





Entrene un k-means con 3, 4 y 5 clusters. ¿Las agrupaciones son lógicas usando un análisis visual?, Grafique los clusters con el código dispuesto más abajo y comente cual de las opciones presenta una mejor agrupación.

En base al análisis realizado, ubique la posición de los cluster en la cancha. Solo señale a que posición equivaldría tomando de referencia:

- Ataque
- Medio campistas
- Defensas
- Arquero 

**To-Do**:

- [ ] Escoger un numero de cluster y justificar elección.
- [ ] Comentar si las agrupaciones son lógicas a través de un análisis visual.
- [ ] Graficar los resultados obtenidos y visualizar las agrupaciones obtenidas.


In [47]:
preprocessed_data = preprocessing_transformer.fit_transform(X)

In [48]:
position = df_players.loc[:,['Club_Position']]

In [49]:
from sklearn.cluster import KMeans

kmeans3 = KMeans(n_clusters=3, random_state=10)
kmeans3.fit(preprocessed_data)

kmeans4 = KMeans(n_clusters=4, random_state=20)
kmeans4.fit(preprocessed_data)

kmeans5 = KMeans(n_clusters=5, random_state=30)
kmeans5.fit(preprocessed_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=30, tol=0.0001, verbose=0)

In [50]:
labels3 = kmeans3.labels_
labels4 = kmeans4.labels_
labels5 = kmeans5.labels_

In [51]:
from umap import UMAP

def graficar_clusters(clusters, preprocessed_data = preprocessed_data, posicion = position['Club_Position']):
  # Entreguele a esta función solo los clusters, el resto de parametros debería funcionar por default.
  umap = UMAP()
  projections = umap.fit_transform(preprocessed_data)
  fig = px.scatter(x = projections[:,0], y=projections[:,1], color = clusters, hover_data={'posicion': posicion}) 
  fig.show()

In [52]:
graficar_clusters(labels3)

In [55]:
graficar_clusters(labels4)

In [56]:
graficar_clusters(labels5)

**Respuesta**:

```
El mejor número de clusters según K-Means son tres clusters. En cuanto a la distribución, es evidente según el gráfico que los arqueros se ubican en la posición inferior izquierda muy separados del resto de los jugadores de campo. Se identifica en el sector amarillo del gráfico mayoritariamente defensas, y en el sector rojo se ubican mediocampistas y atacantes de manera conjunta sin un patrón definido. En cambio, para el clustering de 4 centroides es algo más indefinida la distribución de los jugadores de campo, por tanto se prefiere el primer número de clusters.

```

### 1.3.2 Búsqueda Número Óptimo de Clusters: Método del Codo **(Opcional 0.5 puntos)**

<p align="center">
  <img src="https://c.tenor.com/fk4p5HzTi5gAAAAC/elbow-high-five.gif" width="300">
</p>





Renancin muy indeciso de sus resultados le comenta que desea realizar un último trabajo. En esta última tarea le solicita la agrupación de los datos obtenidos en la sección 1.2 utilizando K-means. Para esto usted deberá visualizar a través del método del codo el número óptimo de agrupaciones para los datos. 

**Hint:** Revisar las clases y encontrar el código.

**To-Do**:

- [ ] Pre-procesar los datos con el mismo preprocesamiento aplicado en 1.2
- [ ] Utilizar el método del codo para obtener el número de clusters.

In [57]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=0).fit(preprocessed_data).inertia_]
    for i in range(1, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(3)

,n° clusters,inertia
0,1,222993.693715
1,2,156239.636163
2,3,121709.452505


In [58]:
px.line(intertias, x="n° clusters", y="inertia", title="Método del Codo con K-Means")

**Respuesta**:

```
En base al método del codo, podemos observar que existe un cambio "considerable" en la inertia hasta el número de clusters 2, y esto se ve en nuestro gráfico anterior, donde podemos ver que con 3 clusters no es tan evidente la separación entre todos ellos, pero si habría una separación evidente entre 2 clusters. Además, podrían interpretarse cada uno de los 2 clusters como arqueros y jugadores de campo, donde si existe una diferencia clara.

```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>